In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib
import matplotlib.pyplot as plt
import time
import numpy as np
import query_db
import composite
import spectral_analysis as sa
import scipy.interpolate as inter
from scipy.special import erf

In [2]:
query = ["SELECT * from Supernovae inner join Photometry ON Supernovae.SN = Photometry.SN and phase >= -1 and phase < 1"]
# query = ["SELECT * from Supernovae inner join Photometry ON Supernovae.SN = Photometry.SN where phase >= -.5 and phase <= .5 and ((dm15_source >= 1.1 or dm15_from_fits >= 1.1)) and ((dm15_source <= 1.2 or dm15_from_fits <= 1.2)) and source != 'swift_uv' and  (source = 'cfa' or source='other')"]

In [3]:
composites, sn_arrays, boot_sn_arrays = query_db.main(len(query), query, boot='nb', medmean=1)

SQL Query: SELECT * from Supernovae inner join Photometry ON Supernovae.SN = Photometry.SN and phase >= -1 and phase < 1
164 questionable files currently ignored
Generate variance failed for:  [u'sn2004dt-20040820-hst.flm']
11 questionable spectra removed 199 spectra left
5 Peculiar Ias removed 194 spectra left
149 spectra with host corrections
164 questionable files currently ignored
101 valid SNe found
Arrays cleaned
3 spectra may have telluric contamination
1991bg sn1991bg-19911213-noz.flm 130.242135187 None None 0.681687715271 0.003392 bsnip 3340.0 8602.0 3.0
Host correction...
1994s sn1994s-19940616-uoi.flm 34.8321198485 0.94 None 0.998840596273 0.015177 bsnip 3124.0 11082.0 11.0
Host correction...
1998dk sn1998dk-19980831-ui.flm 138.561375893 1.05 None -0.728371655089 0.013219 bsnip 3308.0 10018.0 14.0
Host correction...
1998es sn1998es-19981125-ui-corrected.flm 107.924025303 0.85 None 0.496750833938 0.010567 bsnip 3346.0 10044.0 8.0
Host correction...
1999aa sn1999aa-19990223-ui

In [4]:
def make_gini_ranges():
    sn_arr = sn_arrays[0]
    gini_ranges = []
    
#     x = np.linspace(1000, 12000, num = 89)
#     x = np.linspace(1000, 12000, num = 45)
    x = np.linspace(1000, 12000, num = 23)
#     x = np.linspace(1000, 12000, num = 12)
    for i in range(len(x)-1):
        gini_ranges.append((x[i], x[i+1]))
    print gini_ranges
    return gini_ranges

In [5]:
def set_SN_gweights(sn_arr, gini_ranges):
    for SN in sn_arr:
        g_weights = []
        for g_r in gini_ranges:
            g_locs = np.where((SN.wavelength >= g_r[0]) & (SN.wavelength < g_r[1]))[0]
            g_w = np.sum(SN.ivar[g_locs])
            if g_w == 0.:
                g_w = np.nan
            g_weights.append(g_w)
        SN.g_array = g_weights

In [6]:
def calc_gini_coeffs(sn_arr):
    num_specs = []
    for i in range(len(sn_arr[0].g_array)):
        num_specs.append(len(sn_arr))
        for SN in sn_arr:
            if np.isnan(SN.g_array[i]):
                num_specs[i] -= 1
    print num_specs
    
    gini_coeffs = []
    for i in range(len(sn_arr[0].g_array)):
        gini_num = 0.
        gini_denom = 0.
        for SNi in sn_arr:
            g_wi = SNi.g_array[i]
            jsum = 0.
            for SNj in sn_arr:
                g_wj = SNj.g_array[i]
                g_diff = np.absolute(g_wi - g_wj)
                jsum = np.nansum([jsum, g_diff])
            gini_num = np.nansum([gini_num,jsum])
            gini_denom = np.nansum([gini_denom, g_wi])
        gini_denom *= 2.*num_specs[i]
        gini_coeffs.append(gini_num/gini_denom)
    return gini_coeffs, num_specs

In [7]:
gini_ranges = make_gini_ranges()

[(1000.0, 1500.0), (1500.0, 2000.0), (2000.0, 2500.0), (2500.0, 3000.0), (3000.0, 3500.0), (3500.0, 4000.0), (4000.0, 4500.0), (4500.0, 5000.0), (5000.0, 5500.0), (5500.0, 6000.0), (6000.0, 6500.0), (6500.0, 7000.0), (7000.0, 7500.0), (7500.0, 8000.0), (8000.0, 8500.0), (8500.0, 9000.0), (9000.0, 9500.0), (9500.0, 10000.0), (10000.0, 10500.0), (10500.0, 11000.0), (11000.0, 11500.0), (11500.0, 12000.0)]


In [8]:
set_SN_gweights(sn_arrays[0], gini_ranges)

In [9]:
gini_coeffs, num_specs = calc_gini_coeffs(sn_arrays[0])

[0, 8, 8, 10, 50, 93, 96, 97, 98, 98, 97, 97, 89, 43, 42, 42, 37, 24, 20, 2, 1, 0]


In [10]:
for i in range(len(gini_coeffs)):
    print gini_ranges[i], gini_cokeffs[i], num_specs[i]

(1000.0, 1500.0) nan 0
(1500.0, 2000.0) 0.764569630174 8
(2000.0, 2500.0) 0.741463431665 8
(2500.0, 3000.0) 0.78650834316 10
(3000.0, 3500.0) 0.912501338959 50
(3500.0, 4000.0) 0.87872468774 93
(4000.0, 4500.0) 0.823716363365 96
(4500.0, 5000.0) 0.769619289756 97
(5000.0, 5500.0) 0.758736483987 98
(5500.0, 6000.0) 0.767570866357 98
(6000.0, 6500.0) 0.765976854877 97
(6500.0, 7000.0) 0.792322225279 97
(7000.0, 7500.0) 0.795597602284 89
(7500.0, 8000.0) 0.656826044834 43
(8000.0, 8500.0) 0.653615082487 42
(8500.0, 9000.0) 0.679468314216 42
(9000.0, 9500.0) 0.689682997868 37
(9500.0, 10000.0) 0.794125938187 24
(10000.0, 10500.0) 0.8559241254 20
(10500.0, 11000.0) 0.47553502061 2
(11000.0, 11500.0) 0.0 1
(11500.0, 12000.0) nan 0


In [11]:
sn_arr = sn_arrays[0]
deweight_spec = []
deweight_ranges = []
replace_ivar_sns = []
for i in range(len(gini_coeffs)):
    if gini_coeffs[i] >= .7:
        g_list = []
        for j, SN in enumerate(sn_arr):
            g_list.append(SN.g_array[i])
        for k, g in enumerate(g_list):
            if np.isnan(g_list[k]):
                g_list[k] = 0.
        g_sort = np.argsort(g_list)
        print np.sort(g_list)
#         print sn_arr[g_sort[-1]].name, sn_arr[g_sort[-2]].name
        replace_ivar_sns.append(sn_arr[g_sort[-2]])
        max_ind = np.nanargmax(g_list)
        deweight_spec.append(sn_arr[max_ind])
        deweight_ranges.append(gini_ranges[i])

[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00

In [12]:
for i, dSN in enumerate(deweight_spec):
    g_r = deweight_ranges[i]
    g_locs = np.where((SN.wavelength >= g_r[0]) & (SN.wavelength < g_r[1]))[0]
    print g_r
#     dSN.ivar[glocs] = replace_ivar_sns.ivar[glocs]

(1500.0, 2000.0)
(2000.0, 2500.0)
(2500.0, 3000.0)
(3000.0, 3500.0)
(3500.0, 4000.0)
(4000.0, 4500.0)
(4500.0, 5000.0)
(5000.0, 5500.0)
(5500.0, 6000.0)
(6000.0, 6500.0)
(6500.0, 7000.0)
(7000.0, 7500.0)
(9500.0, 10000.0)
(10000.0, 10500.0)
